<a href="https://colab.research.google.com/github/maverick98/CDS/blob/main/DNN_Assignment2a_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group No 189

## Group Member Names:
• MAHESH NARAM (2023aa05876): 100%

• GIRIJA SHANKAR SAHOO (023AA05235): 100%

• SOURAJEET SAHOO (023aa05029): 100%

• MANO RANJAN SAHU (2023aa05738): 100%
## Journal used for the implementation
Journal title: Wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations  
Authors: Alexei Baevski, Henry Zhou, Abdelrahman Mohamed, Michael Auli  
Journal Name: NeurIPS  
Year: 2020


# 1. Import the required libraries

In [1]:
##---------Type the code below this line------------------##
##---------Type the code below this line------------------##
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import librosa
import numpy as np


# 2. Data Acquisition

For the problem identified by you, students have to find the data source themselves from any data source.

Provide the URL of the data used.

Write Code for converting the above downloaded data into a form suitable for DL



In [2]:
##---------Type the code below this line------------------##
##---------Type the code below this line------------------##

# Dataset: LibriSpeech
# URL: https://www.openslr.org/12

# Loading a sample audio file for demonstration
audio_file = "sample_audio.wav"
data, sampling_rate = librosa.load(audio_file, sr=16000)  # loading audio at 16 kHz


<ipython-input-2-df55807605ef>:9: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sampling_rate = librosa.load(audio_file, sr=16000)  # loading audio at 16 kHz
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'sample_audio.wav'

# 3. Data Preparation

Perform the data prepracessing that is required for the data that you have downloaded.


This stage depends on the dataset that is used.

In [ ]:
##---------Type the code below this line------------------##

# Process the audio data using the Wav2Vec2 processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
input_values = processor(data, return_tensors="pt", sampling_rate=16000).input_values

# Splitting into train/test sets if necessary (demonstration purposes)
train_data, test_data = input_values[:80%], input_values[80%:]

##---------Type the code below this line------------------##

## Split the data into training set and testing set
##---------Type the code below this line------------------##

## Identify the target variables.
##---------Type the code below this line------------------##


## 4. Deep Neural Network Architecture


## 4.1 Design the architecture that you will be using

* CNN / RNN / Transformer as per the journal referenced



In [ ]:
##---------Type the code below this line------------------##
##---------Type the code below this line------------------##

class Wav2Vec2ForSpeechRecognition(nn.Module):
    def __init__(self):
        super(Wav2Vec2ForSpeechRecognition, self).__init__()
        # Load Wav2Vec2 model for feature extraction
        self.wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        # Linear layer for final classification
        self.classifier = nn.Linear(self.wav2vec.config.hidden_size, 29)  # 29 for character-level classification

    def forward(self, input_values):
        # Extract features from the Wav2Vec2 model
        features = self.wav2vec(input_values).last_hidden_state
        # Pass through classifier
        logits = self.classifier(features)
        return logits


## 4.2 DNN Report

Report the following and provide justification for the same.

* Number of layers
* Number of units in each layer
* Total number of trainable parameters



# 5. Training the model


In [ ]:
# Configure the training, by using appropriate optimizers, regularizations and loss functions
##---------Type the code below this line------------------##
##---------Type the code below this line------------------##

# Model initialization
model = Wav2Vec2ForSpeechRecognition()

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_function = nn.CTCLoss()  # Connectionist Temporal Classification for speech

# Training loop (simplified)
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(train_data)
    loss = loss_function(logits, target)  # Assuming `target` contains the correct labels
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


# 6. Test the model


In [ ]:
##---------Type the code below this line------------------##
##---------Type the code below this line------------------##

model.eval()
with torch.no_grad():
    logits = model(test_data)
    # Evaluate the model using test data and report accuracy
    predictions = torch.argmax(logits, dim=-1)
    print("Test accuracy:", compute_accuracy(predictions, test_labels))  # Assume test_labels contain ground truth


# 7. Report the result

1. Plot the training and validation accuracy history.
2. Plot the training and validation loss history.
3. Report the testing accuracy and loss.
4. Show Confusion Matrix for testing dataset.
5. Report values for preformance study metrics like accuracy, precision, recall, F1 Score.


In [ ]:
##---------Type the code below this line------------------##
# Example code to plot accuracy
import matplotlib.pyplot as plt

plt.plot(training_accuracy, label="Training Accuracy")
plt.plot(validation_accuracy, label="Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy History')
plt.legend()
plt.show()


### NOTE


All Late Submissions will incur a <b>penalty of -2 marks </b>. So submit your assignments on time.

Good Luck